In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
EPS = 1e-15
from torch.nn import Parameter
from torch_sparse import SparseTensor
from torch_geometric.utils.num_nodes import maybe_num_nodes
import pandas as pd
import numpy as np
import torch_cluster  
random_walk = torch.ops.torch_cluster.random_walk
device = 'cpu'
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
from torch_geometric.datasets import Planetoid
beta = 4.5
#gama = 0.5

def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    print("AP： ",average_precision_score(true_edges, predict_edges))
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("预测准确率： ",accuracy_score(true_edges, predict_edges))
    #print("预测精确率： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("召回率： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))
    
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, ne, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        NE = ne
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, cc):
        CC = cc
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.9)
        #self.loss_function = torch.nn.L1Loss(reduction='sum')
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, cc, edge_train, edge_test):
        CC = cc
        GP = g
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach()
    
####################################### 以最大化预测效果为目标的类型质心更新 ########################################
class ClusteringLayer(nn.Module):
    """给定节点嵌入，更新类簇质心"""
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, x):
        NE = x
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(ne, ini_cc, g, edge_train, edge_test, num_epochs=5):        
    NE = ne
    CC = ini_cc
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=0.4, momentum=0.9)
    #loss_function = torch.nn.L1Loss(reduction='sum')
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach()

In [2]:
def get_cora_graph(filename_adj, nodes_number):
    datasets = Planetoid('./data', filename_adj)
    edges = datasets[0].edge_index
    
    #print("总边数： ", edges.shape[1])
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[1]):
        graph_np[edges[0,i], edges[1,i]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [3]:
filename = "Citeseer"    # 节点索引从0开始, sep=None
nodes_number = 3327
train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)

In [4]:
len(train_edge),len(test_edge)

(20480, 912)

In [5]:
embedding_dim = 10
n_clusters = 24
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.6169829111665015
AUC SCORE:  0.6118372960911049
AP：  0.6144894163664709
AUC SCORE:  0.6092066789781472
######################### 模块循环 ： 1 ##########################
AP：  0.6298071651365537
AUC SCORE:  0.6185941828254848
AP：  0.6254793209565012
AUC SCORE:  0.6126356186518929
######################### 模块循环 ： 2 ##########################
AP：  0.6361070826768463
AUC SCORE:  0.6204360957217605
AP：  0.6306507177800299
AUC SCORE:  0.6116305016928285
######################### 模块循环 ： 3 ##########################
AP：  0.6313966742931835
AUC SCORE:  0.6068790397045245
AP：  0.6437502991829829
AUC SCORE:  0.6261397737765466
######################### 模块循环 ： 4 ##########################
AP：  0.6429996278539039
AUC SCORE:  0.6213642659279779
AP：  0.6386623210058177
AUC SCORE:  0.6144631040320099
######################### 模块循环 ： 5 ##########################
AP：  0.6416175857478028
AUC SCORE:  0.6193299861495846
AP：  0.6379944082992174

AP：  0.8087425110802313
AUC SCORE:  0.7396314250538627
######################### 模块循环 ： 48 ##########################
AP：  0.8093790573343653
AUC SCORE:  0.7406461603570329
AP：  0.8094236356607742
AUC SCORE:  0.7407519621421976
######################### 模块循环 ： 49 ##########################
AP：  0.8102257714076155
AUC SCORE:  0.7421514312096029
AP：  0.8106465691491569
AUC SCORE:  0.7429305170821792
######################### 模块循环 ： 50 ##########################
AP：  0.8114124678851788
AUC SCORE:  0.7444838796552785
AP：  0.811731035710409
AUC SCORE:  0.7452196829793782
######################### 模块循环 ： 51 ##########################
AP：  0.812449257347618
AUC SCORE:  0.7463642659279779
AP：  0.8126131094178541
AUC SCORE:  0.7468548014773777
######################### 模块循环 ： 52 ##########################
AP：  0.8133511714124373
AUC SCORE:  0.7478743459526007
AP：  0.8138450639611035
AUC SCORE:  0.7488145390889505
######################### 模块循环 ： 53 ##########################
AP：  0.814574421696

AP：  0.8394554533801196
AUC SCORE:  0.7948791935980302
AP：  0.839880098113575
AUC SCORE:  0.7960911049553709
######################### 模块循环 ： 96 ##########################
AP：  0.8395912217511019
AUC SCORE:  0.7951677439212065
AP：  0.8398837190078936
AUC SCORE:  0.7961872883964296
######################### 模块循环 ： 97 ##########################
AP：  0.8395696086440745
AUC SCORE:  0.7952446906740536
AP：  0.840093087275805
AUC SCORE:  0.7965335487842413
######################### 模块循环 ： 98 ##########################
AP：  0.8396251998742918
AUC SCORE:  0.7951388888888888
AP：  0.8401197068341464
AUC SCORE:  0.7964517928593413
######################### 模块循环 ： 99 ##########################
AP：  0.8398456025382783
AUC SCORE:  0.7955284318251771
AP：  0.8404644693419505
AUC SCORE:  0.7971298861188058
######################### 模块循环 ： 100 ##########################
AP：  0.8399786862849421
AUC SCORE:  0.7956823253308711
AP：  0.8405845718721685
AUC SCORE:  0.7973991997537704
######################### 

AP：  0.8466707373786
AUC SCORE:  0.8074359418282548
######################### 模块循环 ： 143 ##########################
AP：  0.8458157544701334
AUC SCORE:  0.8057046398891966
AP：  0.846827447638522
AUC SCORE:  0.8076523545706371
######################### 模块循环 ： 144 ##########################
AP：  0.8462698836090552
AUC SCORE:  0.8060989919975377
AP：  0.8469596862503455
AUC SCORE:  0.8076138811942136
######################### 模块循环 ： 145 ##########################
AP：  0.8464509825000341
AUC SCORE:  0.8064789165897199
AP：  0.8469671164744152
AUC SCORE:  0.8076860187750077
######################### 模块循环 ： 146 ##########################
AP：  0.8465558988597288
AUC SCORE:  0.8069021237303786
AP：  0.846579726357013
AUC SCORE:  0.8069934979993845
######################### 模块循环 ： 147 ##########################
AP：  0.8465640185281367
AUC SCORE:  0.8065847183748845
AP：  0.8460769537345527
AUC SCORE:  0.8053872345337025
######################### 模块循环 ： 148 ##########################
AP：  0.846847576

AP：  0.8519610564944554
AUC SCORE:  0.8136590104647583
AP：  0.8523399563327289
AUC SCORE:  0.8148324484456756
######################### 模块循环 ： 191 ##########################
AP：  0.8516898754870726
AUC SCORE:  0.8130049630655586
AP：  0.8500764356740292
AUC SCORE:  0.8103118267159125
######################### 模块循环 ： 192 ##########################
AP：  0.8499502328040622
AUC SCORE:  0.8116535857186826
AP：  0.8502887373046882
AUC SCORE:  0.8119950369344414
######################### 模块循环 ： 193 ##########################
AP：  0.8507088510402878
AUC SCORE:  0.8122932056017236
AP：  0.850927728705595
AUC SCORE:  0.8127741228070176
######################### 模块循环 ： 194 ##########################
AP：  0.8505193271967434
AUC SCORE:  0.8111822868574946
AP：  0.8507780172215484
AUC SCORE:  0.811706486611265
######################### 模块循环 ： 195 ##########################
AP：  0.8501363897254061
AUC SCORE:  0.810513811942136
AP：  0.8505740363872283
AUC SCORE:  0.811538165589412
########################

...

AP：  0.8604331530810377
AUC SCORE:  0.8340402816251153
AP：  0.8604415565171903
AUC SCORE:  0.8346221914435211
######################### 模块循环 ： 571 ##########################
AP：  0.8609960936628637
AUC SCORE:  0.8352810480147737
AP：  0.8605213890725123
AUC SCORE:  0.8351512003693445
######################### 模块循环 ： 572 ##########################
AP：  0.8611169989081784
AUC SCORE:  0.8356513542628502
AP：  0.8608644659530745
AUC SCORE:  0.8355936441982148
######################### 模块循环 ： 573 ##########################
AP：  0.8614923572299238
AUC SCORE:  0.8367430363188674
AP：  0.8614559628998306
AUC SCORE:  0.8374403662665435
######################### 模块循环 ： 574 ##########################
AP：  0.8610610766329116
AUC SCORE:  0.835420514004309
AP：  0.8613905271137543
AUC SCORE:  0.8360168513388735
######################### 模块循环 ： 575 ##########################
AP：  0.8618815884017819
AUC SCORE:  0.8362525007694676
AP：  0.8577503915665604
AUC SCORE:  0.8291204986149585
#####################

AP：  0.8653722336024721
AUC SCORE:  0.8427496922129886
######################### 模块循环 ： 618 ##########################
AP：  0.8657222616527516
AUC SCORE:  0.842797783933518
AP：  0.864512772007942
AUC SCORE:  0.841124192059095
######################### 模块循环 ： 619 ##########################
AP：  0.8635864061670191
AUC SCORE:  0.8407779316712835
AP：  0.861806233528125
AUC SCORE:  0.8355311249615266
######################### 模块循环 ： 620 ##########################
AP：  0.8671058116785932
AUC SCORE:  0.8432835103108649
AP：  0.8672164305551734
AUC SCORE:  0.8451975607879347
######################### 模块循环 ： 621 ##########################
AP：  0.8659904180846545
AUC SCORE:  0.8415522083718067
AP：  0.8664988437268051
AUC SCORE:  0.8430045783317943
######################### 模块循环 ： 622 ##########################
AP：  0.8674437951826326
AUC SCORE:  0.8449474838411819
AP：  0.8675652857708435
AUC SCORE:  0.8455871037242229
######################### 模块循环 ： 623 ##########################
AP：  0.86741018

AP：  0.8628713484504896
AUC SCORE:  0.8368969298245614
AP：  0.8631084752757974
AUC SCORE:  0.8376279239766082
######################### 模块循环 ： 666 ##########################
AP：  0.8629014801073729
AUC SCORE:  0.8370460141582026
AP：  0.86317667068583
AUC SCORE:  0.8376038781163435
######################### 模块循环 ： 667 ##########################
AP：  0.863030382903075
AUC SCORE:  0.8371806709756848
AP：  0.8633663117752132
AUC SCORE:  0.8378010541705141
######################### 模块循环 ： 668 ##########################
AP：  0.8634584420400815
AUC SCORE:  0.8377144890735612
AP：  0.8639038031254855
AUC SCORE:  0.8383012080640195
######################### 模块循环 ： 669 ##########################
AP：  0.8645232470651034
AUC SCORE:  0.8395035010772545
AP：  0.8646752065600867
AUC SCORE:  0.8397583871960602
######################### 模块循环 ： 670 ##########################
AP：  0.8651021720807343
AUC SCORE:  0.8405182363804247
AP：  0.8650161023288784
AUC SCORE:  0.8405711372730071
#######################

AP：  0.8629984598581218
AUC SCORE:  0.8365650969529086
######################### 模块循环 ： 713 ##########################
AP：  0.8628205484960982
AUC SCORE:  0.8368103647276084
AP：  0.8636480223902885
AUC SCORE:  0.837954947676208
######################### 模块循环 ： 714 ##########################
AP：  0.8627558271584709
AUC SCORE:  0.8366228070175439
AP：  0.8627851616821747
AUC SCORE:  0.8363823484148969
######################### 模块循环 ： 715 ##########################
AP：  0.8628408907805868
AUC SCORE:  0.8365891428131732
AP：  0.8633739557920228
AUC SCORE:  0.8372816635887965
######################### 模块循环 ： 716 ##########################
AP：  0.8629546499850458
AUC SCORE:  0.8367430363188673
AP：  0.8631361894698107
AUC SCORE:  0.8370363958140967
######################### 模块循环 ： 717 ##########################
AP：  0.8629735400844354
AUC SCORE:  0.8369402123730378
AP：  0.8627955365616975
AUC SCORE:  0.8368632656201909
######################### 模块循环 ： 718 ##########################
AP：  0.86213

AP：  0.8632898471966016
AUC SCORE:  0.8365939519852261
AP：  0.8639090203178224
AUC SCORE:  0.8379693751923669
######################### 模块循环 ： 761 ##########################
AP：  0.8638994402712546
AUC SCORE:  0.837651969836873
AP：  0.8637772857597299
AUC SCORE:  0.8373538011695906
######################### 模块循环 ： 762 ##########################
AP：  0.8639638409283813
AUC SCORE:  0.837954947676208
AP：  0.8644714404959736
AUC SCORE:  0.8393592259156664
######################### 模块循环 ： 763 ##########################
AP：  0.8636492553261269
AUC SCORE:  0.8375750230840258
AP：  0.8637093435064077
AUC SCORE:  0.8378972376115728
######################### 模块循环 ： 764 ##########################
AP：  0.8635230958363617
AUC SCORE:  0.8374788396429671
AP：  0.8638820818914391
AUC SCORE:  0.8383060172360726
######################### 模块循环 ： 765 ##########################
AP：  0.8636308461824179
AUC SCORE:  0.8374115112342259
AP：  0.8631395451478279
AUC SCORE:  0.8371277700831026
######################